## Raw Data Ingestion & Parsing

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/raw/superstore_sales_clean.csv")
df.head()

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408.0,2.0,0.0,106.140,35.46,Medium,2011.0
1,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.0,3.0,0.1,36.036,9.72,Medium,2011.0
2,HU-2011-1220,1/1/2011,5/1/2011,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Office Supplies,Storage,"Tenex Box, Single Width",66.0,4.0,0.0,29.640,8.17,High,2011.0
3,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,...,Office Supplies,Paper,"Enermax Note Cards, Premium",45.0,3.0,0.5,-26.055,4.82,High,2011.0
4,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",114.0,5.0,0.1,37.770,4.70,Medium,2011.0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        51290 non-null  object 
 1   order_date      51290 non-null  object 
 2   ship_date       51290 non-null  object 
 3   ship_mode       51290 non-null  object 
 4   customer_name   51290 non-null  object 
 5   segment         51290 non-null  object 
 6   state           51290 non-null  object 
 7   country         51290 non-null  object 
 8   market          51290 non-null  object 
 9   region          51290 non-null  object 
 10  product_id      51290 non-null  object 
 11  category        51290 non-null  object 
 12  sub_category    51290 non-null  object 
 13  product_name    51290 non-null  object 
 14  sales           51290 non-null  float64
 15  quantity        51290 non-null  float64
 16  discount        51290 non-null  float64
 17  profit          51290 non-null 

## Date Parsing & Type Conversion

In [3]:
# Converting 'order_date', 'ship_date' to datetime format with dayfirst=True and year to int
df['order_date'] = pd.to_datetime(
    df['order_date'],
    format='mixed',
    dayfirst=True
)

df['ship_date'] = pd.to_datetime(
    df['ship_date'],
    format='mixed',
    dayfirst=True
)

df['year'] = df['year'].astype(int)

## De-duplication

In [4]:
# Remove duplicates
df = df.drop_duplicates()

## Numeric Coercion & Critical Null Handling

In [5]:
# Convert numeric fields except quantity
numeric_cols = ['sales', 'discount', 'profit', 'shipping_cost']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Convert quantity to integer
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce').astype('Int64')

# Drop rows with invalid sales, profit, or quantity
df = df.dropna(subset=['sales', 'profit', 'quantity'])

In [ ]:
# Feature engineering
# Time features
df['order_year'] = df['order_date'].dt.year
df['order_month'] = df['order_date'].dt.month
df['order_month_name'] = df['order_date'].dt.month_name()
df['order_quarter'] = df['order_date'].dt.to_period('Q').astype(str)

## Operational Feature Engineering (Row-Level)

In [6]:
# Operational metrics
df['shipping_days'] = (df['ship_date'] - df['order_date']).dt.days

## Profit Margin (Row-Level Metric)

In [7]:
# Financial metrics
df['profit_margin'] = np.where(
    df['sales'] != 0,
    df['profit'] / df['sales'],
    0
)

## Discount Banding (Business Logic)

In [8]:
# Discount buckets (important for BI insights)
df['discount_band'] = pd.cut(
    df['discount'],
    bins=[-0.01, 0, 0.2, 0.4, 1],
    labels=['No Discount', 'Low (≤20%)', 'Medium (20–40%)', 'High (>40%)']
)

In [9]:
df.to_csv("data/processed/fact_sales_clean.csv", index=False)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        51290 non-null  object        
 1   order_date      51290 non-null  datetime64[ns]
 2   ship_date       51290 non-null  datetime64[ns]
 3   ship_mode       51290 non-null  object        
 4   customer_name   51290 non-null  object        
 5   segment         51290 non-null  object        
 6   state           51290 non-null  object        
 7   country         51290 non-null  object        
 8   market          51290 non-null  object        
 9   region          51290 non-null  object        
 10  product_id      51290 non-null  object        
 11  category        51290 non-null  object        
 12  sub_category    51290 non-null  object        
 13  product_name    51290 non-null  object        
 14  sales           51290 non-null  float64       
 15  qu

In [13]:
df['state'].nunique()

1094

In [20]:
# Missing values
df.isna().sum().sort_values(ascending=False)

# Negative profit rate
negative_profit_rate = (df['profit'] < 0).mean() * 100
negative_profit_rate

np.float64(24.455059465782803)

In [21]:
kpis = {
    "Total Sales": df['sales'].sum(),
    "Total Profit": df['profit'].sum(),
    "Average Profit Margin": df['profit_margin'].mean(),
    "Total Orders": df['order_id'].nunique(),
    "Total Customers": df['customer_name'].nunique(),
    "Avg Shipping Days": df['shipping_days'].mean()
}

pd.DataFrame.from_dict(kpis, orient='index', columns=['Value'])

,Value
Total Sales,1.264290e+07
Total Profit,1.469035e+06
Average Profit Margin,4.720706e-02
Total Orders,2.503500e+04
Total Customers,7.950000e+02
Avg Shipping Days,3.969370e+00


In [22]:
monthly_trend = (
    df.groupby(df['order_date'].dt.to_period('M'))
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum')
      )
      .reset_index()
)

monthly_trend['order_date'] = monthly_trend['order_date'].astype(str)
monthly_trend.head()

,order_date,total_sales,total_profit
0,2011-01,98902.0,8321.80096
1,2011-02,91152.0,12417.90698
2,2011-03,145726.0,15303.56826
3,2011-04,116927.0,12902.32438
4,2011-05,146762.0,12183.82870


In [23]:
category_perf = (
    df.groupby('category')
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_margin=('profit_margin', 'mean')
      )
      .sort_values('total_sales', ascending=False)
)

category_perf


,total_sales,total_profit,avg_margin
category,,,
Technology,4744691.0,663778.73318,0.049658
Furniture,4110884.0,286782.25380,0.008748
Office Supplies,3787330.0,518473.83430,0.058557


In [24]:
discount_analysis = (
    df.groupby('discount_band', observed=True)
      .agg(
          orders=('order_id', 'count'),
          revenue=('sales', 'sum'),
          profit=('profit', 'sum'),
          avg_margin=('profit_margin', 'mean')
      )
      .reset_index()
)

discount_analysis

,discount_band,orders,revenue,profit,avg_margin
0,No Discount,29009,6992734.0,1.770695e+06,0.264827
1,Low (≤20%),10953,3719929.0,5.130216e+05,0.152060
2,Medium (20–40%),4367,1083907.0,-1.872711e+05,-0.200076
3,High (>40%),6961,846335.0,-6.274110e+05,-0.869543


In [25]:
customer_summary = (
    df.groupby('customer_name')
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          orders=('order_id', 'nunique'),
          avg_discount=('discount', 'mean')
      )
      .reset_index()
)
customer_summary.head()

,customer_name,total_sales,total_profit,orders,avg_discount
0,Aaron Bergman,24646.0,4683.20800,37,0.110112
1,Aaron Hawkins,20759.0,2450.92904,34,0.160929
2,Aaron Smayling,14207.0,369.16180,31,0.167667
3,Adam Bellavance,20189.0,4979.97690,41,0.131765
4,Adam Hart,21720.0,1902.03342,42,0.093238


In [ ]:
# Save for BI
df.to_csv("cleaned_superstore_sales.csv", index=False)